In [4]:
# Essentials
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors
from matplotlib.colors import ListedColormap, BoundaryNorm, TwoSlopeNorm
import matplotlib.ticker as ticker
from matplotlib import patches
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from scipy.interpolate import RegularGridInterpolator

### Load the data and quality control --> only use flag 1 data.

In [6]:
# Load ADCP dataset
adcp1 = xr.open_dataset("/Users/jessicafranks/Desktop/grad_school/grad_thesis/thesis-python/grady-adcp/adcp1_final.nc")

# Apply quality control: Flag == 1
adcp1_qc = adcp1.where(adcp1.Flag == 1)

#### Inspect binned distance

In [8]:
adcp1_qc.bindist

<xarray.DataArray 'bindist' (bindist: 47)>
array([ 1.25,  1.5 ,  1.75,  2.  ,  2.25,  2.5 ,  2.75,  3.  ,  3.25,  3.5 ,
        3.75,  4.  ,  4.25,  4.5 ,  4.75,  5.  ,  5.25,  5.5 ,  5.75,  6.  ,
        6.25,  6.5 ,  6.75,  7.  ,  7.25,  7.5 ,  7.75,  8.  ,  8.25,  8.5 ,
        8.75,  9.  ,  9.25,  9.5 ,  9.75, 10.  , 10.25, 10.5 , 10.75, 11.  ,
       11.25, 11.5 , 11.75, 12.  , 12.25, 12.5 , 12.75])
Coordinates:
  * bindist  (bindist) float64 1.25 1.5 1.75 2.0 2.25 ... 12.0 12.25 12.5 12.75
Attributes:
    units:        m
    description:  Height above the seafloor

### Reynold's number calculations

In [ ]:
kv = 1.22e-6 # m^2/s^-1 this is the kinematic velocity of seawater (Grady 2024)

# Grady 2024
max_current = 0.025 # m/s
min_current = 0.007 # m/s
re = (max_current * diameter) / kv # to be calculated later once I get diameter values for stephy in the field 

### Dispersal model setup

In [21]:
n_particles = 1000
release_height = 3.0
plant_depth = 12.75
sinking_speed = 0.000356
bottom_roughness = 0.08
Kz = 1e-4
dt = 1000
settle_fraction = 0.9

wave_height = 0.5
wave_period = 8.0
g = 9.81

L = (g * wave_period**2) / (2 * np.pi) * np.tanh((2 * np.pi * plant_depth) / (wave_period**2 * g))
k = 2 * np.pi / L
U_s = (np.pi * wave_height**2) / (2 * wave_period * np.sinh(k * plant_depth)**2)

def stokes_drift(z):
    return U_s * np.exp(-2 * k * z)

In [23]:
# Load ADCP and prepare interpolation
ds = xr.open_dataset("/Users/jessicafranks/Desktop/grad_school/grad_thesis/thesis-python/grady-adcp/adcp1_final.nc")

valid = ds.Flag == 1
east = ds.East.where(valid)
north = ds.North.where(valid)

bindist = ds.bindist.values
time = ds.time.values
time_sec = (time - time[0]) / np.timedelta64(1, 's')

u_data = east.transpose("time", "bindist").values
v_data = north.transpose("time", "bindist").values

u_interp = RegularGridInterpolator((time_sec, bindist), u_data, bounds_error=False, fill_value=0)
v_interp = RegularGridInterpolator((time_sec, bindist), v_data, bounds_error=False, fill_value=0)


In [25]:
# Propagule definitions
class Particle:
    def __init__(self, z=release_height):
        self.x = 0.0
        self.y = 0.0
        self.z = z
        self.settled = False

In [27]:
# Run simulation Function
def run_simulation(release_height, plant_depth):
    particles = [Particle(z=release_height) for _ in range(n_particles)]
    positions = []
    t = 0
    settled_count = 0
    max_time_steps = len(time_sec)
    
    while settled_count < settle_fraction * n_particles and t < max_time_steps - 1:
        t_sec = time_sec[t]
        settled_count = 0
        current_positions = []

        for p in particles:
            if not p.settled:
                u = u_interp((t_sec, p.z))
                v = v_interp((t_sec, p.z))
                u += stokes_drift(p.z)

                p.x += u * dt
                p.y += v * dt

                dz_sink = sinking_speed * dt
                dz_diff = np.random.normal(0, np.sqrt(2 * Kz * dt))
                p.z -= dz_sink
                p.z += dz_diff

                if p.z <= bottom_roughness:
                    p.z = 0
                    p.settled = True
                    settled_count += 1

            current_positions.append((p.x, p.y, p.z))

        positions.append(current_positions)
        t += 1

    return np.array(positions[-1]) if positions else np.array([])

In [ ]:
# Run simulations
release_heights = [1, 3, 5]
plant_depths = [12.0, 12.75]

results = {}

for r in release_heights:
    for pd in plant_depths:
        label = f"r{r}_pd{pd}"
        trajectories = run_simulation(release_height=r, plant_depth=pd)
        results[label] = trajectories

In [ ]:
# Plot final particle positions
final = results['r3_pd12.75']
x, y, z = final[:, 0], final[:, 1], final[:, 2]

plt.figure(figsize=(8,6))
plt.scatter(x, y, c=z, cmap='viridis')
plt.colorbar(label="Height above bottom (m)")
plt.xlabel("Eastward Displacement (m)")
plt.ylabel("Northward Displacement (m)")
plt.title("Final Particle Positions")
plt.grid()
plt.show()

In [ ]:
# Dispersal Kernel density pfloatfrom scipy.stats import gaussian_kde

final_x = x
final_y = y

xy = np.vstack([final_x, final_y])
kde = gaussian_kde(xy)

xgrid, ygrid = np.meshgrid(
    np.linspace(final_x.min()-10, final_x.max()+10, 100),
    np.linspace(final_y.min()-10, final_y.max()+10, 100)
)
positions_kde = np.vstack([xgrid.ravel(), ygrid.ravel()])
density = kde(positions_kde).reshape(xgrid.shape)

plt.figure(figsize=(8,6))
plt.contourf(xgrid, ygrid, density, cmap='viridis')
plt.colorbar(label="Kernel density")
plt.scatter(final_x, final_y, c='red', s